<a href="https://colab.research.google.com/github/muhammetsnts/SPARK/blob/main/1.Spark_Basic_Concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyspark

# Creating and Reading DB Table From a File

In [ ]:
df = sqlContext.sql("SELECT * FROM newtable")

In [ ]:
df.show()

# Spark DataFrame Basics

## Start Spark Session

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# start spark session

spark = SparkSession.builder.appName("Basics").getOrCreate()

In [ ]:
df.printSchema()

root
-- age: long (nullable = true)
-- name: string (nullable = true)

In [ ]:
df.columns

Out[55]: ['age', 'name']

In [ ]:
df.describe()

Out[56]: DataFrame[summary: string, age: string, name: string]

In [ ]:
df.describe().show()

+-------+------------------+-------+
summary| age| name|
+-------+------------------+-------+
 count| 2| 3|
 mean| 24.5| null|
 stddev|7.7781745930520225| null|
 min| 19| Andy|
 max| 30|Michael|
+-------+------------------+-------+

## Creating Schema

In [ ]:
df = spark.read.json("people.json")

In [ ]:
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [ ]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
# burada schema nasıl ouşturulur onu gösteriyoruz. örneğin indexi integer, jsl_ner_labelı string yaptık. 
# True parametresi null değer görünce hata vermemesi için. bişey yazmazsak veya False yaparsak "buradaki değer null" diyerek hata verir.

data_schema = [StructField("age", IntegerType(), True),
               StructField("name", StringType(), True)]

In [ ]:
final_struct = StructType(fields = data_schema)

In [ ]:
# şimdi dosyayı okurken schemanın final_struct ta belirtildiği gibi olmasını istiyoruz

df = spark.read.json("/FileStore/tables/people.json", schema=final_struct)
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [ ]:
df.printSchema()

root
-- age: integer (nullable = true)
-- name: string (nullable = true)

## Type of the Return Values

In [ ]:
# returns column

df['age']

In [ ]:
type(df['age'])

Out[69]: pyspark.sql.column.Column

In [ ]:
# returns DataFrame

df.select("age")

Out[70]: DataFrame[age: int]

In [ ]:
type(df.select("age"))

Out[72]: pyspark.sql.dataframe.DataFrame

In [ ]:
df.select("age").show()

+----+
 age|
+----+
null|
 30|
 19|
+----+

In [ ]:
df.head(2)

Out[73]: [Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [ ]:
df.head(2)[0]

Out[74]: Row(age=None, name='Michael')

In [ ]:
# row type

type(df.head(2)[0])

Out[76]: pyspark.sql.types.Row

## Select Multiple Column

In [ ]:
df.select(["age", "name"]).show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

## Create New Column

In [ ]:
# adding a new column or replacing an existing column

df.withColumn("double_age", df["age"]*2).show()

+----+-------+----------+
 age| name|double_age|
+----+-------+----------+
null|Michael| null|
 30| Andy| 60|
 19| Justin| 38|
+----+-------+----------+

In [ ]:
# not an inplace operation

df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

## Rename Column

In [ ]:
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
my_new_age| name|
+----------+-------+
 null|Michael|
 30| Andy|
 19| Justin|
+----------+-------+

In [ ]:
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

## Temporary View

We can register our dataframe as a sql temporary view and run sql queries on it.

In [ ]:
# we registered df as sql temp view named people

df.createOrReplaceTempView('people')

In [ ]:
# now we can run sql queries

results = spark.sql("SELECT * FROM people")
results.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [ ]:
new_res = spark.sql("SELECT * FROM people WHERE age=30")
new_res.show()

+---+----+
age|name|
+---+----+
 30|Andy|
+---+----+